In [1]:
# 필요한 라이브러리
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm
from scipy.linalg import eig

In [2]:
# 한글 폰트 설정
font_path = 'C:/Windows/Fonts/malgun.ttf'
fontprop = fm.FontProperties(fname=font_path)

plt.rc('font', family=fontprop.get_name())

# Unicode 마이너스 설정
plt.rcParams['axes.unicode_minus'] = False

# 1. 쌍대 비교 행렬 생성

In [6]:
pairwise_matrix = np.array([
    [1, 38.89/22.22, 38.89/38.89],  # Average Freight vs others
    [22.22/38.89, 1, 22.22/38.89],  # Average Delivery Time vs others
    [38.89/38.89, 38.89/22.22, 1]   # Order Volume vs others
])

# 요인명
factors = ['average_freight', 'average_delivery_time', 'order_volume']

# 2. 고유벡터 (중요도, 가중치) 계산

In [7]:
# 각 행을 합계로 나누어 정규화 (각 열의 비율의 합계가 1이 되도록)
sum_columns = pairwise_matrix.sum(axis=0)
normalized_matrix = pairwise_matrix / sum_columns

# 각 행의 평균을 계산하여 중요도 도출
weights = normalized_matrix.mean(axis=1)

# 3. 일관성 비율(CR) 계산 및 일관성 판단

In [8]:
# 최대 고유값 계산
lambda_max = np.dot(sum_columns, weights)

# 일관성 지수(CI) 계산
n = pairwise_matrix.shape[0]
ci = (lambda_max - n) / (n - 1)

# 무작위 지수(RI) 값 (n=3인 경우 약 0.58)
ri = 0.58

# 일관성 비율(CR) 계산
cr = ci / ri

# 일관성 체크
if cr < 0.1:
    consistency_message = f"일관성 비율 CR = {cr:.4f}: 일관성 있음."
else:
    consistency_message = f"일관성 비율 CR = {cr:.4f}: 일관성 부족. 재검토 필요."

# 결과 출력 (가중치와 일관성 체크 결과)
print("요인별 가중치:")
for factor, weight in zip(factors, weights):
    print(f"{factor}: {weight:.4f}")

print("\n", consistency_message)

요인별 가중치:
average_freight: 0.3889
average_delivery_time: 0.2222
order_volume: 0.3889

 일관성 비율 CR = 0.0000: 일관성 있음.


# 4. 가중치 기반 점수 산출

In [10]:
# Olist 데이터 생성 (총 매출에서 차지하는 비율이 5% 미만인 주 제외)
data = {
    'state': ['SP', 'RJ', 'MG', 'BA', 'RS', 'PR', 'SC'],
    'average_freight': [15.15, 20.95, 20.62, 26.35, 21.70, 20.52, 21.50],
    'average_delivery_time': [210.3, 367.4, 288.0, 463.5, 367.2, 287.3, 358.9],
    'order_volume': [5161905, 1809997, 1572574, 506107, 742311, 676198, 518578]
}

df = pd.DataFrame(data)

# 요인 값 정규화
df['Normalized Freight'] = df['average_freight'] / df['average_freight'].max()
df['Normalized Delivery Time'] = df['average_delivery_time'] / df['average_delivery_time'].max()
df['Normalized Order Volume'] = df['order_volume'] / df['order_volume'].max()

# 가중치를 적용한 종합 점수 계산 (weights는 배열 =>인덱스로 접근)
df['Score'] = (
    weights[0] * df['Normalized Freight'] +
    weights[1] * df['Normalized Delivery Time'] +
    weights[2] * df['Normalized Order Volume']
)

# 가장 높은 점수를 받은 주를 순위별로 정렬하여 출력
df['Rank'] = df['Score'].rank(ascending=False)

# 순위별로 정렬된 결과를 출력
df = df[['state', 'Score', 'Rank']].sort_values(by='Rank')
df

,state,Score,Rank
0,SP,0.713316,1.0
3,BA,0.649230,2.0
1,RJ,0.621697,3.0
2,MG,0.560875,4.0
4,RS,0.552231,5.0
6,SC,0.528444,6.0
5,PR,0.491530,7.0


- 결과 해석  
AHP분석의 점수 결과는 0~1사이의 숫자로 표시된다. 따라서 일반적으로 0.5~1사이의 점수를 가진 선택안을 채택한다.  
SP(상파울루주)가 0.71의 높은 점수로 최적 입지로 선정되었다.  
2위 BA(바이아주), 3위 RJ(리우데자네이루주)도 지형적 위치와 지역 발달 정도를 고려해 볼 필요가 있음.